CARGA DE LOS ARCHIVOS CSV

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

common_player_info = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\common_player_info.csv')
draft_combine_stats = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\draft_combine_stats.csv')
draft_history = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\draft_history.csv')
game = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\game.csv')
game_info = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\game_info.csv')
game_summary = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\game_summary.csv')
inactive_players = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\inactive_players.csv')
line_score = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\line_score.csv')
officials = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\officials.csv')
other_stats = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\other_stats.csv')
player = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\player.csv')
play_by_play = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\play_by_play.csv')
team = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\team.csv')
team_details = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\team_details.csv')
team_history = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\team_history.csv')
team_info_common = pd.read_csv(r'C:\Users\matis\OneDrive\Documentos\NBA dataset\csv\team_info_common.csv')

In [5]:
import pandas as pd
import pyodbc

# Configurar la conexión a SQL Server (autenticación de Windows)
server = r'Asus_Mati\SQLEXPRESS'
database = 'DAFT10_PF'
conn = pyodbc.connect(f'DRIVER=SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes')

# Función para limpiar y preparar los datos
def limpiar_datos(df):
    # Reemplazar valores no válidos y ajustar precisión
    for col in df.select_dtypes(include=['float']):
        df[col] = pd.to_numeric(df[col], errors='coerce').round(2)  # Ajustar decimales
    df.fillna(value=0, inplace=True)  # Reemplazar NaN con 0 (o un valor adecuado)
    return df

# Función para cargar datos a SQL Server
def cargar_datos(csv_file, table_name, conn):
    try:
        # Cargar el CSV en un DataFrame
        df = pd.read_csv(csv_file)
        df = limpiar_datos(df)

        # Preparar SQL para inserción en bloque
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['?' for _ in df.columns])
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

        # Inserción en bloque
        data = [tuple(row) for row in df.to_numpy()]
        cursor = conn.cursor()
        cursor.executemany(sql, data)
        conn.commit()
        cursor.close()
        print(f"Datos cargados en {table_name} correctamente.")
    except Exception as e:
        print(f"Error al cargar datos en {table_name}: {e}")

# Cargar cada archivo CSV en su tabla correspondiente
archivos = {
    "common_player_info.csv": "common_player_info",
    "draft_combine_stats.csv": "draft_combine_stats",
    "draft_history.csv": "draft_history",
    "officials.csv": "officials",
    "player.csv": "player",
    "inactive_players.csv": "inactive_players",
    "game.csv": "game",
    "game_info.csv": "game_info",
    "game_summary.csv": "game_summary",
    "line_score.csv": "line_score",
    "other_stats.csv": "other_stats",
    "team.csv": "team",
    "team_details.csv": "team_details",
    "team_history.csv": "team_history"
}

# Iterar sobre los archivos y cargar los datos
for archivo, tabla in archivos.items():
    cargar_datos(archivo, tabla, conn)

conn.close()


Error al cargar datos en common_player_info: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK__common_p__543848DF81C069FE'. Cannot insert duplicate key in object 'dbo.common_player_info'. The duplicate key value is (76001). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")
Datos cargados en draft_combine_stats correctamente.
Datos cargados en draft_history correctamente.
Datos cargados en officials correctamente.
Datos cargados en player correctamente.
Datos cargados en inactive_players correctamente.
Error al cargar datos en game: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'blk_home'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'pts_home'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)")
Error al cargar

realice la transformacion de datos para que se cargaran correctamente a la base de SQL Server

In [ ]:
import pandas as pd
import pyodbc

# Configurar la conexión a SQL Server
server = r'Asus_Mati\SQLEXPRESS'
database = 'DAFT10_PF'
conn = pyodbc.connect(f'DRIVER=SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes')
#server: Especifica el nombre del servidor SQL Server.
#database: Define el nombre de la base de datos.
#conn: Establece una conexión a SQL Server utilizando pyodbc con autenticación integrada de Windows.


def obtener_columnas_tabla(conn, table_name):
    query = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}'"
    cursor = conn.cursor()
    cursor.execute(query)
    return [row[0] for row in cursor.fetchall()]
# Consulta las columnas de la tabla especificada en la base de datos mediante la vista INFORMATION_SCHEMA.COLUMNS.
# Retorna una lista de nombres de columnas, que será utilizada para filtrar los datos del archivo CSV.


def limpiar_y_preparar_datos(df, columnas_tabla):
    df = df.drop_duplicates()  # Eliminar duplicados
    df = df[columnas_tabla]  # Filtrar solo columnas válidas
    df.fillna(value=0, inplace=True)  # Manejar valores nulos
    return df
#drop_duplicates(): Elimina filas duplicadas del DataFrame.
#df[columnas_tabla]: Selecciona solo las columnas que coinciden con las de la tabla en la base de datos, evitando errores.
#fillna(value=0): Reemplaza valores nulos con 0 para evitar conflictos al insertar datos.

def cargar_datos(csv_file, table_name, conn):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(csv_file)
        columnas_tabla = obtener_columnas_tabla(conn, table_name)
        df = limpiar_y_preparar_datos(df, columnas_tabla)

        # Preparar la consulta SQL
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['?' for _ in df.columns])
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

        # Ejecutar inserción
        data = [tuple(row) for row in df.to_numpy()]
        cursor = conn.cursor()
        cursor.executemany(sql, data)
        conn.commit()
        cursor.close()
        print(f"Datos cargados en {table_name} correctamente.")
    except Exception as e:
        print(f"Error al cargar datos en {table_name}: {e}")

# Carga el archivo CSV en un DataFrame de pandas.
# Filtra las columnas del CSV según las columnas de la tabla SQL.
# Crea una instrucción INSERT INTO dinámica usando los nombres de las columnas y placeholders (?) para los valores.
# Convierte los datos del DataFrame en una lista de tuplas para usar con executemany.
#Inserta los datos en la tabla SQL y confirma los cambios.

# Diccionario de archivos y tablas
archivos = {
    "common_player_info.csv": "common_player_info",
    "draft_combine_stats.csv": "draft_combine_stats",
    "draft_history.csv": "draft_history",
    "officials.csv": "officials",
    "player.csv": "player",
    "inactive_players.csv": "inactive_players",
    "game.csv": "game",
    "game_info.csv": "game_info",
    "game_summary.csv": "game_summary",
    "line_score.csv": "line_score",
    "other_stats.csv": "other_stats",
    "team.csv": "team",
    "team_details.csv": "team_details",
    "team_history.csv": "team_history"
}
#Mapea cada archivo CSV con la tabla correspondiente en la base de datos.

for archivo, tabla in archivos.items():
    cargar_datos(archivo, tabla, conn)
#Itera sobre el diccionario, llamando a la función cargar_datos para cada archivo y tabla.
conn.close()
#Cierra la conexión a la base de datos después de completar la carga de datos.

Datos cargados en common_player_info correctamente.
Datos cargados en draft_combine_stats correctamente.
Datos cargados en draft_history correctamente.
Datos cargados en officials correctamente.
Datos cargados en player correctamente.
Datos cargados en inactive_players correctamente.
Datos cargados en game correctamente.
Datos cargados en game_info correctamente.
Datos cargados en game_summary correctamente.
Datos cargados en line_score correctamente.
Datos cargados en other_stats correctamente.
Datos cargados en team correctamente.
Datos cargados en team_details correctamente.
Datos cargados en team_history correctamente.


Resumen<br>
El script automatiza el proceso de:<br>
<br>
Leer múltiples archivos CSV.<br>
Limpiar y preparar los datos.<br>
Insertarlos en las tablas correspondientes en SQL Server, gestionando posibles errores.<br>